## Install the package dependencies before running this notebook

In [1]:
import numpy 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import pickle
from glob import glob

"""
    number of trajectories in each city
    # austin --  train: 43041 test: 6325 
    # miami -- train: 55029 test:7971
    # pittsburgh -- train: 43544 test: 6361
    # dearborn -- train: 24465 test: 3671
    # washington-dc -- train: 25744 test: 3829
    # palo-alto -- train:  11993 test:1686

    trajectories sampled at 10HZ rate, input 5 seconds, output 6 seconds
    
"""

'\n    number of trajectories in each city\n    # austin --  train: 43041 test: 6325 \n    # miami -- train: 55029 test:7971\n    # pittsburgh -- train: 43544 test: 6361\n    # dearborn -- train: 24465 test: 3671\n    # washington-dc -- train: 25744 test: 3829\n    # palo-alto -- train:  11993 test:1686\n\n    trajectories sampled at 10HZ rate, input 5 seconds, output 6 seconds\n    \n'

In [2]:
os.getcwd()

'/Users/lianglin/Desktop/CSE151/argo2'

## Create a Torch.Dataset class for the training dataset

In [3]:
from glob import glob
import pickle
import numpy as np

ROOT_PATH = os.getcwd() + "/"

cities = ["austin", "miami", "pittsburgh", "dearborn", "washington-dc", "palo-alto"]
splits = ["train", "test"]

# Get data as numpy array.
# If it's train data, return the input and output (for validation)
# If it's test data, return only the input (for prediction)
def get_city_trajectories(city="palo-alto", split="train", normalized=False):
    f_in = ROOT_PATH + split + "/" + city + "_inputs"
    inputs = pickle.load(open(f_in, "rb"))
    inputs = np.asarray(inputs)
    
    outputs = None
    
    if split=="train":
        f_out = ROOT_PATH + split + "/" + city + "_outputs"
        outputs = pickle.load(open(f_out, "rb"))
        outputs = np.asarray(outputs)

    return inputs, outputs


class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, city: str, split:str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.transform = transform

        self.inputs, self.outputs = get_city_trajectories(city=city, split=split, normalized=False)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):

        data = (self.inputs[idx], self.outputs[idx])
            
        if self.transform:
            data = self.transform(data)

        return data

# intialize a dataset
city = 'palo-alto' 
split = 'train'
train_dataset  = ArgoverseDataset(city = city, split = split)

In [4]:
x, y = train_dataset[0]
print(x.shape)

(50, 2)


## LSTM Model

In [5]:
class LSTM(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(self.input_size, self.hidden_size, self.num_layers, batch_first=True)
        self.fc = nn.Softmax(dim=1)
    
    def forward(self, x, hidden, cell):
        out, (hidden, cell) = self.lstm(x, (hidden, cell))
        print("out = ", out)
        print("hidden = ", hidden)
        print("cell = ", cell)
        return out, (hidden, cell)
    
    def init_hidden(self, batch_size):
#         hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size)
#         cell = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        cell = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        return hidden, cell

In [6]:
class generator:
    
    def __init__(self):
        pass
    
    def to_tensor(self, array):
        pass
    
    def get_batch(self):
        pass
    
    def generate(self):
        pass
    
    def train(self):
        pass

## Create a DataLoader class for training

In [7]:
batch_sz = 4  # batch size 
train_loader = DataLoader(train_dataset,batch_size=batch_sz)

## Sample a batch of data and visualize 

In [8]:
import matplotlib.pyplot as plt
import random

model = LSTM(2, 128, 1)

def show_sample_batch(sample_batch):
    """visualize the trajectory for a batch of samples"""
    inp, out = sample_batch
    batch_sz = inp.size(0)
    agent_sz = inp.size(1)
    
    fig, axs = plt.subplots(1,batch_sz, figsize=(15, 3), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .5, wspace=.001)
    axs = axs.ravel()   
    for i in range(batch_sz):
        axs[i].xaxis.set_ticks([])
        axs[i].yaxis.set_ticks([])
        
        # first two feature dimensions are (x,y) positions
        axs[i].scatter(inp[i,:,0], inp[i,:,1])
        axs[i].scatter(out[i,:,0], out[i,:,1])
        
for i_batch, sample_batch in enumerate(train_loader):
    inp, out = sample_batch
    inp = inp.float()
    inp = pack_padded_sequence(inp, torch.tensor([50 for i in range(batch_sz)]), batch_first=True)
    """
    TODO:
      implement your Deep learning model
      implement training routine
    """
    hidden, cell = model.init_hidden(batch_size=batch_sz)
    model.zero_grad()
    loss = 0
    output, (hidden, cell) = model(inp, hidden, cell)
    
#     show_sample_batch(sample_batch)
    break

out =  PackedSequence(data=tensor([[ 7.6159e-01, -3.9393e-10, -6.3114e-04,  ...,  7.6123e-01,
          2.8951e-29, -7.6144e-01],
        [ 7.6159e-01, -2.3026e-07, -2.6690e-03,  ...,  2.2361e-02,
          5.7620e-24, -7.6159e-01],
        [ 7.6159e-01, -2.0391e-04, -2.0104e-03,  ...,  2.7939e-18,
          1.3545e-27, -7.6159e-01],
        ...,
        [ 7.6159e-01, -1.1639e-05, -3.5599e-03,  ...,  9.6167e-01,
          5.8291e-25, -1.0000e+00],
        [ 1.0000e+00, -7.6255e-03, -1.8012e-03,  ...,  2.4434e-16,
          1.1729e-28, -1.0000e+00],
        [ 1.0000e+00, -1.1177e-11, -6.1328e-07,  ...,  2.3013e-22,
          0.0000e+00, -1.0000e+00]], grad_fn=<CatBackward0>), batch_sizes=tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4]), sorted_indices=None, unsorted_indices=None)
hidden =  tensor([[[ 7.6159e-01, -2.7609e-08, -7.3971e-04, -1.0000e+00,  0.0000e+00,